# **TP 2 Hidden Markov Model** 
## E-Mail Segmentation
### _**Abdennour Kerboua**_

### Task : automatic segmentation of mails, problem statement

$\renewcommand{\P}{\mathbb{P}}$

### **Question 1**


An e-mail necessarily starts with the header. It follows : $\P(q_0 = \text{header}) = 1$ and then $\P(q_0 = \text{body}) = 1 - \P(q_0 = \text{header}) = 0$

We can then compute $\pi$ : $$\pi = \begin{bmatrix}\P(q_0 = \text{header}) \\ \P(q_0 = \text{body}) \end{bmatrix} = \begin{bmatrix}1\\ 0\end{bmatrix} $$

### **Question 2**

The transition matrix is given by : $$A =\begin{bmatrix} 0.999218078035812 & 0.000781921964187974 \\ 0 & 1 \end{bmatrix}$$

Hence, the probability of switching from the first state to the second is given by : $$P(2 \mid 1) = A_{12} = 0.000781921964187974 $$ and of staying in state 2 by : $$P(2 \mid 2) = A_{22} = 1$$ The highest probability is the one of staying in the state 2 ($P(2\mid 2) = 1$) which makes sense : reaching the body means the header has ended so we inevitably stay in the body. The lowest probability is the one of switching from the second to the first which is null for the same reason explained before ($P(1\mid 2) = A_{21} =  0$).


### **Question 3**

Let $N$ be the number of different characters. 

The observation matrix given by $$B = [P(c_i,s)]_{(i,s) \in \llbracket 1,N\rrbracket \times \{0,1\}}$$ of size $N \times 2$ (column $s$ gives the distribution of characters knowing the position in the state $s$).

In [1]:
import numpy as np

In [ ]:
# Load parameters
B = np.loadtxt('distribution/P.text')
A = np.array([[0.999218, 0.000782],[0, 1]])
pi = np.array([1,0])

# Load data
mails = []
for i in range(1,31):
    mails.append(np.loadtxt('dat/mail{}.dat'.format(i)).astype(int))

In [155]:
def Viterbi(obs, A, B, pi):
    Q = len(A)
    T = len(obs)
    small = 1e-10
    A = np.log(A + small)
    B = np.log(B + small)
    pi = np.log(pi + small)
    delta = np.zeros((Q, T))
    psi = np.zeros((Q, T))
    q = np.zeros(T)
    delta[:,0] = pi + B[obs[0],:]
    for t in range(1,T):
        for j in range(Q):
            delta[j,t] = np.max(delta[:,t-1] + A[:,j]) + B[obs[t],j]
            psi[j,t] = np.argmax(delta[:,t-1] + A[:,j])
    q[T-1] = np.argmax(delta[:,T-1])
    for t in range(T-2,-1,-1):
        q[t] = psi[int(q[t+1]),t+1]
    return q

In [156]:
q = []
for mail in mails:
    q.append(Viterbi(mail, A, B, pi))

In [175]:
def visualize_segmentation(mail_filename, visualized_mail_filename, path):
	## @parameter mail_filename : Path to the mail on wich we try the algorithm.
	## @parameter visualized_mail_filename : The path on which we write the mail with the v 	
	# ## @parameter path : The sequence of 0 and 1 that the Viterbi algorithm returns.
    ## return: True if the header corresponds to the
	i=0
	while path[i] == 0:
	    i+=1
	visu = open(visualized_mail_filename, 'w') 
	if(mail_filename.endswith(".dat")):
		mail_filename = mail_filename[:-4] + ".txt" 
	mail = open(mail_filename, 'r', encoding='utf-8', errors='replace')
	header = mail.read(i)
	visu.write(header) 
	visu.write("\n===================== cut here\n") 
	visu.write(mail.read(int(path.sum())))
	visu.close() 
	mail.close() 
	return i

In [177]:
import os
if not os.path.exists('result'):
    os.makedirs('result')

for i in range(30):
   j = visualize_segmentation('dat/mail{}.dat'.format(i+1), 'result/mail{}_viterbi.txt'.format(i+1), q[i])
   print("File mail{}_viterbi.txt has been created".format(i+1)+"cut at index {}".format(j))

File mail1_viterbi.txt has been createdcut at index 2735
File mail2_viterbi.txt has been createdcut at index 2445
File mail3_viterbi.txt has been createdcut at index 2263
File mail4_viterbi.txt has been createdcut at index 1914
File mail5_viterbi.txt has been createdcut at index 2302
File mail6_viterbi.txt has been createdcut at index 2436
File mail7_viterbi.txt has been createdcut at index 2555
File mail8_viterbi.txt has been createdcut at index 2305
File mail9_viterbi.txt has been createdcut at index 2540
File mail10_viterbi.txt has been createdcut at index 1886
File mail11_viterbi.txt has been createdcut at index 1776
File mail12_viterbi.txt has been createdcut at index 1952
File mail13_viterbi.txt has been createdcut at index 2304
File mail14_viterbi.txt has been createdcut at index 3539
File mail15_viterbi.txt has been createdcut at index 2183
File mail16_viterbi.txt has been createdcut at index 1971
File mail17_viterbi.txt has been createdcut at index 2282
File mail18_viterbi.txt

### **Question 4** Print the track and present and discuss the results obtained on mail11.txt to mail30.txt.

We are going to take as example the mail11.txt and mail30.txt files.

- For email 11, the segmentation is done at the 2850th character. If we look at the real email, we can see that the header ends at the 2851th character, so the segmentation is almost correct.

- For email 30, the segmentation is done at the 2172th character. If we look at the real email, we can see that the header ends at the 2250th character. Even if the segmentation is not perfect, it is still very close to the real value.


### **Question 5** How would you model the problem if you had to segment the mails in more than two parts (for example : header, body, signature)? Draw a diagram of the corresponding Hidden Markov model and give an example of A matrix that would be suitable in this case.

Let's take the header, body and signature as the three states of the model. The transition matrix $A$ would be:

$$A = \begin{pmatrix} a_{11} & a_{12} & a_{13} \\ 0 & a_{22} & a_{23} \\ 0 & 0 & 1 \end{pmatrix}$$

As we can see, it is impossible to move from the body to the header or the signature, and it is impossible to move from the signature to the header or the body.
The value of $a_{13}$ would be non-zero if we want to allow the possibility of moving from the header to the signature, otherwise it would be 0.

The initial probabilities vector $\pi$ would be:

$$\pi = \begin{bmatrix} 1 \\ 0 \\ 0 \end{bmatrix}$$

The emission matrix $B$ would be a $256 \times 3$ matrix, because we have 3 states and 256 possible observations.
